In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from geopandas import GeoDataFrame
from sklearn import preprocessing

plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.labelsize'] = 12

#### No estoy seguro si esta bien, pero pruebo

In [2]:
propiedades = pd.read_csv('properati.csv')

In [43]:
def obtener_barrio(fila):
    barrio,padre = fila
    return padre.split('|')[3]

In [44]:
propiedades.loc[:, 'place_name'] = propiedades.loc[:, ['place_name','place_with_parent_names']].apply(lambda x: obtener_barrio(x), axis = 1)

In [6]:
tipo_propiedad = {'PH': [0], 'apartment':[587], 'house': [1829], 'store': [28531]}

In [9]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipo_propiedad[x]) 

In [45]:
import random
barrios = {}
x = 0
for i in propiedades.groupby('place_name').agg(np.mean).lat.keys():
    if i in barrios:
        continue
    barrios[i] = [x]
    x+=random.randint(10, 100)

In [14]:
propiedades.reset_index(drop = True, inplace = True)

In [46]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: barrios[x]) 

In [74]:
adivinar = propiedades.loc[propiedades.mes >= 7,:]

In [75]:
entrenar = propiedades.loc[propiedades.mes < 7,:]

In [50]:
from sklearn import preprocessing
tipos = preprocessing.OneHotEncoder()
tipos.fit([[0], [587], [1829], [28531]]) 

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [53]:
preparacion = barrios.values()
for i in range (0, len(barrios)):
    preparacion[i] = preparacion[i]

In [54]:
pre_barrios = preprocessing.OneHotEncoder()
pre_barrios.fit(preparacion)

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [56]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: pre_barrios.transform([x]))

In [57]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipos.transform([x]))

In [77]:
condiciones = []
precios = []

for i in range(0,len(entrenar)):
    if np.isnan(entrenar.price_aprox_usd[i]):
        continue
    if np.isinf(entrenar.ditancia_subtes[i]):
        continue
    l = list(entrenar.place_name[i].toarray()[0]) + list(entrenar.property_type[i].toarray()[0])
    l.append(entrenar.ditancia_subtes[i])
    l.append(entrenar.seguridad[i])
    l.append(entrenar.gimnasio[i])
    condiciones.append(l)
    precios.append(entrenar.price_aprox_usd[i])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


In [79]:
#propiedades.place_name[0].toarray()[0]
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.9,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [80]:
def adivinar_precio_indice(fila, estimador):
    barrio,tipo,subte, seg, gim = fila
    l =  list(barrio.toarray()[0]) + list(tipo.toarray()[0])
    if np.isinf(subte):
        return 0 # Ver de resolver estos despues
    l.append(subte)
    l.append(seg)
    l.append(gim)
    return  estimador.predict([l])[0]

In [81]:
adivinar.loc[:,'precio_adivinado_gradient'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, est), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [82]:
adivinar.loc[:,'error_porcentual_gradient'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [106]:
adivinar.loc[adivinar.precio_adivinado_gradient>1000,:].error_porcentual_gradient.describe()

count    16205.000000
mean        83.544287
std        201.700593
min          0.000000
25%         18.903710
50%         43.096393
75%         79.090711
max       9038.270013
Name: error_porcentual_gradient, dtype: float64

In [92]:
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [94]:
adivinar.loc[:,'precio_adivinado_gradient_01'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, est), axis = 1)

In [95]:
adivinar.loc[:,'error_porcentual_gradient_01'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient_01']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [105]:
adivinar.loc[adivinar.precio_adivinado_gradient_01>1000,:].error_porcentual_gradient_01.describe()

count    1.645800e+04
mean     6.942295e+01
std      1.631405e+02
min      8.623357e-14
25%      1.738087e+01
50%      3.798468e+01
75%      6.916166e+01
max      5.788523e+03
Name: error_porcentual_gradient_01, dtype: float64

In [108]:
est = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.9,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [109]:
adivinar.loc[:,'precio_adivinado_gradient_10000'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, est), axis = 1)

In [110]:
adivinar.loc[:,'error_porcentual_gradient_10000'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient_10000']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [112]:
adivinar.loc[adivinar.precio_adivinado_gradient_10000>1000,:].error_porcentual_gradient_10000.describe()

count    16181.000000
mean        83.795200
std        201.972879
min          0.000000
25%         18.898221
50%         43.257067
75%         79.274035
max       9033.411137
Name: error_porcentual_gradient_10000, dtype: float64

## Con SVR

In [113]:
from sklearn import svm

In [114]:
clf = svm.SVR(degree = 2)
clf.fit(condiciones, precios)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [116]:
adivinar.loc[:,'precio_adivinado_svr'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x,clf), axis = 1)

In [117]:
adivinar.loc[:,'error_porcentual_svr'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_svr']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [121]:
adivinar.loc[adivinar.precio_adivinado_svr != 0].error_porcentual_svr.describe()

count    16478.000000
mean        59.056386
std         96.720346
min          0.000235
25%         22.490996
50%         46.370952
75%         73.992968
max       2967.125531
Name: error_porcentual_svr, dtype: float64

In [ ]:
# Le cambio el parametro degree y uso la funcion kernel. Creo que que si no esta seteado en poly
# degree no es usado y por lo tanto estaria innecesariamente 
clf = svm.SVR(kernel = 'poly', degree = 5)
clf.fit(condiciones, precios)

In [ ]:
adivinar.loc[:,'precio_adivinado_svr_5'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x,clf), axis = 1)

In [ ]:
adivinar.loc[:,'error_porcentual_svr_5'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_svr_5']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [ ]:
adivinar.loc[adivinar.precio_adivinado_svr_5 != 0].error_porcentual_svr_5.describe()